In [1]:
import cv2
import mediapipe as mp

#얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection #얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils #얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# model_selection = (0)카메라 2m이내의 거리 or (1)5m이내의 거리 
# min_detection_confidence=얼굴일 신뢰도
with mp_face_detection.FaceDetection(model_selection=8, min_detection_confidence=0.1) as face_detection:
      while cap.isOpened(): #동영상 열림 확인
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 검출된 사람 얼굴에 드로우
        if results.detections:
            for detection in results.detections:
                # 6개의 얼굴 특징 : 우눈, 좌눈, 코끝, 입중심, 우귀, 좌귀 
                #mp_drawing.draw_detection(image,detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                nose_tip = keypoints[2] # 코끝

                
                h, w , _ = image.shape #heught, width, channel
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

                # 코 중심으로 모자이크
                a=cv2.circle(image, nose_tip,200, (0,0,0), 1, cv2.LINE_AA)
        
        image=cv2.GaussianBlur(a, (9,9),sigmaX=10)  #가우시안 블러 효과 
        image = cv2.resize(image, None, fx=0.5, fy=0.5)
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection',image)
        
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
#cv2.destroyAllWindows()